In [1]:
from __future__ import annotations

import json
import cProfile
import pstats
from asyncio import sleep

In [2]:
from py_research.db import (
    DataSource,
    TableMap,
    SubTableMap,
    SubMap,
    SelIndex,
    DataBase
)

In [3]:
from test_db_schema import Search, Project, Task, User, Membership, Organization

In [4]:
async def dummy_loader(name: str) -> dict:
    await sleep(0.2)
    return {
        'name': name,
        'age': 20
    }

In [5]:
data_source = DataSource(
    target=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": SubTableMap(
            target=Search.results,
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": SubTableMap(
                    target=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": SubTableMap(
                            target=Task.assignees,
                            push=User.name,
                            match_by=User.name,
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": SubTableMap(
                    target=Project.members,
                    loader=dummy_loader,
                    push={User.name, User.age},
                    rel_map=TableMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                    },
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [6]:
with open("./nested_data.json") as f:
    data = json.load(f)

In [8]:
with cProfile.Profile() as pr:
  db = DataBase()
  rec = await data_source.load([data], db=db)

  stats = pstats.Stats(pr)


Async-loading `Search`: 100%|██████████| 1/1 [00:00<00:00,  5.09it/s]
                                                                     
Async-loading `Project`:  33%|███▎      | 1/3 [00:36<01:13, 36.53s/it]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\lworm\AppData\Local\Temp\ipykernel_3360\3418091003.py:3 in <module>                     │
│                                                                                                  │
│   1 with cProfile.Profile() as pr:                                                               │
│   2   db = DataBase()                                                                            │
│ ❱ 3   rec = await data_source.load([data], db=db)                                                │
│   4                                                                                              │
│   5   stats = pstats.Stats(pr)                                                                   │
│   6                                                                                              │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:855 in load                   │
│                                                                                                  │
│   852 │   │   db = db if db is not None else DataBase()                                          │
│   853 │   │   in_data: InData = {((), ()): dat for dat in data}                                  │
│   854 │   │   rest_data: RestData = []                                                           │
│ ❱ 855 │   │   loaded = await _load_records(                                                      │
│   856 │   │   │   db, cast(TableMap[Record, TreeNode], self), in_data, rest_data                 │
│   857 │   │   )                                                                                  │
│   858                                                                                            │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:807 in _load_records          │
│                                                                                                  │
│   804 │   for rel_tgt, sub_data in lazy_data:                                                    │
│   805 │   │   if isinstance(rel_tgt, SubTableMap):                                               │
│   806 │   │   │   # Full record relation set                                                     │
│ ❱ 807 │   │   │   rec_set[rel_tgt.target] |= await _load_records(                                │
│   808 │   │   │   │   db,                                                                        │
│   809 │   │   │   │   rel_tgt,                                                                   │
│   810 │   │   │   │   cast(InData, sub_data),                                                    │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:766 in _load_records          │
│                                                                                                  │
│   763 │   records: dict[Hashable, Record | Hashable] = {}                                        │
│   764 │   rest_records: InData = {}                                                              │
│   765 │                                                                                          │
│ ❱ 766 │   async for ((parent_idx, path_idx), data), (rec, rel) in loaded:                        │
│   767 │   │   if rec is None:                                                                    │
│   768 │   │   │   rest_records[(parent_idx, path_idx)] = data                                    │
│   769 │   │   │   continue                                                                       │
╰────────────────────────────────────────────────────────────

In [ ]:
db[Project].values()

[]

In [ ]:
print(db[Search].select_str)

SELECT "test_db_schema_Search".term AS term,
       "test_db_schema_Search".result_count AS result_count
FROM "test_db_schema_Search"


In [ ]:
print(db[Search.term].select_str)

SELECT "test_db_schema_Search".term AS term
FROM "test_db_schema_Search"


In [ ]:
print(db[Search.result_count].select_str)

SELECT "test_db_schema_Search".term AS term,
       "test_db_schema_Search".result_count AS result_count
FROM "test_db_schema_Search"


In [ ]:
print(db[Search.results].select_str)

SELECT "test_db_schema_Search".term AS term,
       "test_db_schema_Project".number AS number,
       "test_db_schema_Project".status AS status,
       "test_db_schema_Project"."end" AS "end",
       "test_db_schema_Project".start AS
START, "test_db_schema_Project".name AS name
FROM "test_db_schema_Search"
JOIN "test_db_schema_SearchResult" ON "test_db_schema_SearchResult"."test_db_schema_Search_term" = "test_db_schema_Search".term
JOIN "test_db_schema_Project" ON "test_db_schema_SearchResult"."test_db_schema_Project_number" = "test_db_schema_Project".number


In [ ]:
db[Search.term].values()

['test']

In [ ]:
list(db[Search.result_count].items())

[('test', 3)]

In [ ]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")

         103563 function calls (97694 primitive calls) in 0.253 seconds

   Ordered by: cumulative time
   List reduced from 2069 to 120 due to restriction <'py_research'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.046    0.023 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:4355(_update_dict)
        1    0.000    0.000    0.034    0.034 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:3523(engine)
        2    0.000    0.000    0.024    0.012 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:830(to_df)
    40/25    0.000    0.000    0.020    0.001 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:1694(__get__)
        3    0.000    0.000    0.011    0.004 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:688(_load_records)
       27    0.000    0.000    0.009    0.000 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:2337(_has_t